# **Image Retrieval System**

Web Crawling, Dataset Collection and Annotation

In [2]:
# Install required libraries
!pip install wikipedia
!pip install tqdm
!pip install ultralytics # YOLOv8
!pip install clip #CLIP
!pip install git+https://github.com/openai/clip.git

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=97a6e33fcb061dd11ebcb94ef80a5616b98821f8b0db85d42f3161a60ec3ba34
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.5/974.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 840.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import os
import requests
import wikipedia
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from urllib.request import urlretrieve
from tqdm import tqdm
import time
import csv
from ultralytics import YOLO
import torch
import clip
from PIL import Image

# Load YOLO
model = YOLO('yolov8n.pt')

# Load CLIP model and labels
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)

clip_labels = ["car", "truck", "bus", "motorcycle", "bicycle", "train", "airplane", "boat"]
clip_text_inputs = torch.cat([clip.tokenize(f"a photo of a {label}") for label in clip_labels]).to(device)

# Set up folder for each keyword
def setup_folder(keyword):
    folder_name = keyword.replace(" ", "_") + "_images"
    os.makedirs(folder_name, exist_ok=True)
    return folder_name

# Set up CSV file to save the image-caption lookup table
def setup_csv(keyword):
    csv_filename = keyword.replace(" ", "_") + "_image_captions.csv"
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['image_filename', 'caption', 'detection']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
    return csv_filename

# Get HTML from Wikipedia search result links
def get_wikipedia_links(keyword, max_links=50):
    search_results = wikipedia.search(keyword, results=max_links)
    page_urls = []
    for title in search_results:
        try:
            page = wikipedia.page(title, auto_suggest=False)
            page_urls.append(page.url)
        except Exception as e:
            print(f"Could not get page for '{title}': {e}")
    return page_urls

# Get HTML content from URL
def get_html(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
    except Exception as e:
        print(f"Error fetching URL {url}: {e}")
    return None

def extract_image_urls(html, limit=100, min_size=8000): #8KB
    soup = BeautifulSoup(html, 'html.parser')
    image_data = []

    # Extract images and their alt or title text
    for img in soup.find_all("img"):
        src = img.get("src")
        alt = img.get("alt")  # Get the alt attribute if present
        title = img.get("title")  # Get the title attribute if present

        # Check if the image URL starts with 'upload.wikimedia.org' to ensure it's an image hosted on Wikimedia
        if src and src.startswith("//upload.wikimedia.org"):
            full_url = "https:" + src

            # Skip images with logos or icons by checking the size (based on URL or other heuristics)
            try:
                # Get the image size by making a request to the image URL
                img_response = requests.get(full_url, stream=True)
                img_size = int(img_response.headers.get('Content-Length', 0))  # Image size in bytes

                # Skip if image size is too small (e.g., smaller than 8KB)
                if img_size < min_size:
                    print(f"Skipping small image (too small to be a valid image): {full_url}")
                    continue
            except Exception as e:
                print(f"Could not retrieve image size for {full_url}: {e}")
                continue

            # Skip video, map, gif and other non-image types by checking file extensions in the URL
            if not any(ext in full_url.lower() for ext in ['.jpg', '.jpeg', '.png', '.bmp', '.webp']):
                print(f"Skipping non-image file: {full_url}")
                continue

            # Extract image filename from URL (the last part after the last "/")
            image_filename = os.path.splitext(os.path.basename(full_url))[0]

            # Create the caption by appending the image filename to the alt/title description
            caption = alt if alt else title if title else " "
            caption += f" | {image_filename}"  # Append the filename to the caption

            # Add image to the list if it passes all checks
            image_data.append({
                'url': full_url,
                'captions': caption,  # Captions now include the filename
                'image_filename': image_filename  # Adding image filename
            })

            # Limit the number of images if needed
            if len(image_data) >= limit:
                break

    return image_data


def detect_objects(image_path):
    all_detected_objects = []

    # -------------------- YOLO Detection --------------------
    try:
        results = model(image_path)
        for box in results[0].boxes:
            class_id = int(box.cls)
            class_name = results[0].names[class_id]
            all_detected_objects.append(class_name.lower())
    except Exception as e:
        print(f"YOLO error on {image_path}: {e}")

    # -------------------- CLIP Detection --------------------
    try:
        image = clip_preprocess(Image.open(image_path)).unsqueeze(0).to(device)
        with torch.no_grad():
            image_features = clip_model.encode_image(image)
            text_features = clip_model.encode_text(clip_text_inputs)

            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_features /= text_features.norm(dim=-1, keepdim=True)

            similarity = (100.0 * image_features @ text_features.T).squeeze(0)
            values, indices = similarity.topk(3)  # top 3 results

            clip_results = [clip_labels[i] for i in indices if values[i] > 30]  # confidence threshold
            all_detected_objects.extend(clip_results)
    except Exception as e:
        print(f"CLIP error on {image_path}: {e}")

    return all_detected_objects


# Download images to a folder and save their captions in the CSV file
def download_images(image_data, folder, csv_filename, max_images, current_count):
    with open(csv_filename, 'a', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['image_filename', 'caption', 'detection']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        for data in tqdm(image_data, desc=f"Downloading to {folder}"):
            if current_count >= max_images:
                break
            try:
                image_filename = f"img_{current_count}.jpg"
                file_path = os.path.join(folder, image_filename)  # Make sure to join with the folder path
                urlretrieve(data['url'], file_path)  # Save the image to the specified folder

                # Perform YOLOv8 inference on the image
                try:
                    detected_objects = detect_objects(file_path)
                except Exception as e:
                    print(f"Error processing image {image_filename}: {e}")
                    detected_objects = []  # No objects detected if error occurs

                # Combine caption with detection results
                caption_with_detection = f"{data['captions']} |{', '.join(detected_objects) if detected_objects else ' '}"

                # Save the image and its caption (alt text) to CSV
                writer.writerow({
                    'image_filename': image_filename,
                    'caption': data['captions'],
                    'detection': ', '.join(detected_objects) if detected_objects else ""
                })

                current_count += 1
            except Exception as e:
                print(f"Failed to download {data['url']}: {e}")

    return current_count


# Crawl multiple pages for a keyword
def crawl_keyword_all_pages(keyword, max_images=1000):
    print(f"\nCrawling ALL Wikipedia pages mentioning: {keyword}")
    folder = setup_folder(keyword)
    csv_filename = setup_csv(keyword)
    urls = get_wikipedia_links(keyword, max_links=50)

    total_downloaded = 0
    for url in urls:
        if total_downloaded >= max_images:
            break
        print(f"Processing page: {url}")
        html = get_html(url)
        if html:
            image_data = extract_image_urls(html, limit=200)
            total_downloaded = download_images(image_data, folder, csv_filename, max_images, total_downloaded)
        time.sleep(1)  # Be polite to Wikipedia servers

    print(f"Total downloaded for '{keyword}': {total_downloaded} images")

# Main driver for multiple keywords
def crawl_multiple_keywords_all_pages(keywords, max_images_per_keyword=500):
    for keyword in keywords:
        crawl_keyword_all_pages(keyword, max_images=max_images_per_keyword)

# Add your keywords here
keywords = ["car", "truck", "bus", "motorcycle", "bicycle", "train", "airplane", "boat"]
crawl_multiple_keywords_all_pages(keywords, max_images_per_keyword=500)


100%|███████████████████████████████████████| 338M/338M [00:15<00:00, 22.5MiB/s]



Crawling ALL Wikipedia pages mentioning: car


/usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Could not get page for '.car': ".car" may refer to: 
top-level domain
Content Addressable aRchives
Could not get page for 'Armored car': "Armored car" may refer to: 
Armored car (military)
Armored car (valuables)
Armored car (VIP)
SWAT vehicle
Armored Car (film)
Armored Car (video game)
All pages with titles containing Armored car
All pages with titles containing Armoured car
Could not get page for 'Cable car': "Cable car" may refer to: 
Aerial lift
Aerial tramway
Chairlift
Gondola lift
Bicable gondola lift
Tricable gondola lift
Cable railway
Cable car (railway)
Funicular
Over My Head (Cable Car)
Distant Light
Abhi vs the Universe
Cable ferry
Reaction ferry
Ski lift
Skyline logging
Surface lift
Transporter bridge
Zip-line
Could not get page for 'Car (disambiguation)': "Car (disambiguation)" may refer to: 
Cars (franchise)
Cars (film)
The Car (1977 film)
The Car (1997 film)
BBC Two 1991–2001 idents
The Amazing World of Gumball season 1
The Assistants
an episode of My Hero
Car (magazine)


image 1/1 /content/car_images/img_0.jpg: 352x640 1 person, 2 cars, 1 truck, 315.7ms
Speed: 12.7ms preprocess, 315.7ms inference, 37.2ms postprocess per image at shape (1, 3, 352, 640)


...

CLIP error on boat_images/img_173.jpg: index 7 is out of bounds for dimension 0 with size 3

image 1/1 /content/boat_images/img_174.jpg: 288x640 (no detections), 196.6ms
Speed: 2.6ms preprocess, 196.6ms inference, 1.1ms postprocess per image at shape (1, 3, 288, 640)


CLIP error on boat_images/img_174.jpg: index 7 is out of bounds for dimension 0 with size 3

image 1/1 /content/boat_images/img_175.jpg: 480x640 1 airplane, 1 boat, 196.4ms
Speed: 3.6ms preprocess, 196.4ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)


CLIP error on boat_images/img_175.jpg: index 7 is out of bounds for dimension 0 with size 3


Processing page: https://en.wikipedia.org/wiki/Paper_Boat
Skipping small image (too small to be a valid image): https://upload.wikimedia.org/wikipedia/en/thumb/b/b4/Ambox_important.svg/40px-Ambox_important.svg.png


Total downloaded for 'boat': 176 images


Indexing, Ranking and Retrieval

In [17]:
#!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download("stopwords")
nltk.download("punkt")
nltk.download('punkt_tab')
nltk.download("wordnet")

#Pre-processing
import nltk
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import math

# Ensure the required NLTK resources are downloaded
nltk.download('punkt')  # For tokenization
nltk.download('stopwords')  # For removing stop words (optional)
nltk.download('punkt_tab')
nltk.download("wordnet")

# Initialize stopword list and lemmatizer
STOPWORDS = set(stopwords.words("english"))
LEMMATIZER = WordNetLemmatizer()

# Text Preprocessing Function
def preprocess_text(text):
    allowed_words = ["car", "truck", "bus", "motorcycle", "bicycle", "train", "airplane", "boat"]
    if not text or isinstance(text, float) and math.isnan(text): # Check if text is NaN
        return ""
    text = str(text).lower()  # Convert text to lowercase
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Remove special characters
    text = re.sub(r"<[^>]+>", "", text)  # Remove HTML/XML tags
    text = re.sub(r"\d+", "", text)  # Remove numbers
    words = word_tokenize(text)  # Tokenize (split text into words)
    words = [LEMMATIZER.lemmatize(word) for word in words if word not in STOPWORDS] # Remove Stopwords & Lemmatization
    filtered_words = [word for word in words if word in allowed_words]  # Keep only allowed words
    return " ".join(filtered_words)

# Preprocess the user query
def preprocess_query(query):
    return preprocess_text(query)

# Check if the user query contains any of the predefined classes
def get_class_from_query(query):
    classes = ['bus', 'bicycle', 'car', 'motorcycle', 'airplane', 'train', 'truck', 'boat']
    query = query.lower()  # Convert query to lowercase for easy comparison

    # Check if any class is present in the query
    for vehicle_class in classes:
        if vehicle_class in query:
            return vehicle_class
    return None  # Return None if no class is detected in the query

# Process the CSV and combine caption and detection in the 'caption' column
def preprocess_csv(csv_filename):
    # Load the CSV file into a DataFrame
    df = pd.read_csv(csv_filename)

    # Ensure that the necessary columns exist in the DataFrame
    if 'caption' not in df.columns or 'detection' not in df.columns:
        print(f"CSV file does not contain 'caption' or 'detection' columns.")
        return

    # Apply preprocessing to each row in the CSV for the 'caption' and 'detection' columns
    for index, row in df.iterrows():
        # Preprocess caption and detection
        processed_caption = preprocess_text(row['caption'])
        processed_detection = preprocess_text(row['detection'])

        # Combine both in the 'caption' column
        combined_caption = f"{processed_caption} {processed_detection}".strip()
        df.at[index, 'caption'] = combined_caption  # Update caption with combined text
        df.at[index, 'detection'] = ""  # Clear the 'detection' column as it's now combined in 'caption'

    # Save the preprocessed DataFrame back to CSV
    df.to_csv(csv_filename, index=False)
    print(f"CSV file '{csv_filename}' has been updated with combined caption and detection.")


# Indexing

import pandas as pd
import math
from collections import defaultdict

# Global variables to store the inverted index and document lengths
inverted_index = defaultdict(lambda: defaultdict(int))
doc_lengths = {}

def build_inverted_index_tfidf(csv_filename):
    """
    Builds an inverted index with TF-IDF weights for the 'caption' column in a CSV file.

    Args:
        csv_filename (str): The path to the CSV file.
    """

    print("Building inverted index with TF-IDF...")

    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_filename)

    # Get all unique words from the 'caption' column
    all_words = set()
    for caption in df['caption']:
        # Check if the caption is a string and not NaN before processing
        if isinstance(caption, str):
            words = caption.lower().split()
            all_words.update(words)

    # Calculate IDF for each word
    idf_values = {}
    num_docs = len(df)
    for word in all_words:
        doc_freq = sum(1 for caption in df['caption'] if isinstance(caption, str) and word in caption.lower().split())
        idf_values[word] = math.log(num_docs / (doc_freq + 1))  # Add 1 to avoid division by zero

    # Build the inverted index with TF-IDF weights
    for index, row in df.iterrows():
        doc_id = row['image_filename']  # Using image filename as document ID
        caption = row['caption']  # Get the caption text

        # Check if caption is a string and not NaN before processing
        if isinstance(caption, str):
            caption = caption.lower()  # Get the caption text in lowercase

            words = caption.split()
            doc_lengths[doc_id] = len(words)  # Store document length

            for word in words:
                tf = words.count(word) / len(words)  # Calculate term frequency
                tfidf = tf * idf_values[word]  # Calculate TF-IDF
                inverted_index[word][doc_id] = tfidf  # Store TF-IDF in the inverted index

    print("\nInverted Index with TF-IDF:")
    for word, doc_dict in inverted_index.items():
        print(f"'{word}': {dict(doc_dict)}")



import collections
import math

# Compute BM25
def compute_bm25(query, inverted_index, doc_lengths, k1=1.5, b=0.75):
    """
    Computes BM25 scores for a given query and returns the documents ranked by score.

    Parameters:
    - query: The search query string
    - inverted_index: The inverted index built from captions
    - doc_lengths: The lengths of the documents (captions)
    - k1: BM25 parameter, controls term frequency scaling
    - b: BM25 parameter, controls length normalization

    Returns:
    - A list of tuples (doc_id, score) sorted by score in descending order
    """
    scores = collections.defaultdict(float)
    words = query.split()
    avg_doc_length = sum(doc_lengths.values()) / len(doc_lengths)

    for word in words:
        if word in inverted_index:
            df = len(inverted_index[word])  # Document frequency (number of docs containing the word)
            idf = math.log((len(doc_lengths) - df + 0.5) / (df + 0.5) + 1.0)  # Inverse document frequency (IDF)
            for doc_id, tf in inverted_index[word].items():
                # Term frequency (TF) is normalized based on document length
                norm_tf = tf * (k1 + 1) / (tf + k1 * (1 - b + b * doc_lengths[doc_id] / avg_doc_length))
                scores[doc_id] += idf * norm_tf  # BM25 score is the sum of all terms' contributions


    # Sort the documents by score in descending order and return the top results
    return sorted(scores.items(), key=lambda x: x[1], reverse=True)

# Run retrieval and print top results
def run_retrieval(query_text, inverted_index, doc_lengths, top_n=20):
    """
    Runs BM25 retrieval for a given query and displays top results.

    Parameters:
    - query_text: The search query string
    - inverted_index: The inverted index built from captions
    - doc_lengths: The lengths of the documents (captions)
    - top_n: The number of top results to display

    Returns:
    - None (prints the top results)#returns bm_25 scores
    """
    bm25_scores = compute_bm25(query_text, inverted_index, doc_lengths)[:top_n]

    print(f"Top {top_n} results for query: '{query_text}'")
    for doc_id, score in bm25_scores:
        print(f"Document ID: {doc_id} | Score: {score:.4f}")
    return bm25_scores

# Main driver function
def main(query):
    # Preprocess the user query
    preprocessed_query = preprocess_query(query)

    # Get the corresponding class from the query (e.g., "truck" or None)
    class_name = get_class_from_query(query)

    if class_name:
        # Dynamically construct the CSV filename based on the class name
        csv_filename = f"{class_name}_image_captions.csv"
        print(f"Opening CSV file: {csv_filename}")

        # Preprocess the content of the corresponding CSV
        preprocess_csv(csv_filename)

        build_inverted_index_tfidf(csv_filename)

        bm25_scores = run_retrieval(preprocessed_query, inverted_index, doc_lengths)
        return(bm25_scores,class_name)
    else:
        print(f"No relevant class found in query: {query}")

# Example Usage
#if __name__ == "__main__":
#    user_query = "bus"
#    main(user_query)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Web Interface

In [31]:
!pip install gradio
import gradio as gr
import os

def search_and_display_images(query):
    print(f"Search query: {query}")  # Print the search query for debugging

    # Preprocess the query
    preprocessed_query = preprocess_query(query)

    # Get the class from the query
    class_name = get_class_from_query(query)

    if class_name:
        # Dynamically construct the CSV filename
        csv_filename = f"{class_name}_image_captions.csv"

        # Assuming you've already preprocessed and built the index
        build_inverted_index_tfidf(csv_filename) # If index is not already built

        # Run retrieval
        bm25_scores = compute_bm25(preprocessed_query, inverted_index, doc_lengths)[:10]

        # Get image paths from the top results
        #image_paths = [os.path.join(class_name + "_images", doc_id) for doc_id, _ in bm25_scores]
        #print(image_paths)  # Print the image paths for debugging
        #return image_paths # Return image paths for display in Gradio
        image_paths = []
        for doc_id, _ in bm25_scores:
            image_path = os.path.join(class_name + "_images", doc_id)
            # Check if the image file exists and is not empty
            if os.path.exists(image_path) and os.path.getsize(image_path) > 0:
                image_paths.append(image_path)
                if len(image_paths) == 10:  # Get up to 5 valid images
                    break
            else:
                print(f"Image not found or empty: {image_path}")  # Log the issue

        print(image_paths)  # Print the image paths for debugging
        return image_paths # Return image paths for display in Gradio
    else:
        return ["No relevant class found in query."] # Return error message for Gradio

# Create the Gradio interface
iface = gr.Interface(
    fn=search_and_display_images,
    inputs=gr.Textbox(lines=1, placeholder="Enter your search query..."),
    outputs=gr.Gallery(label="Top 10 Results"),
    title="Wheels & Wings: Search for Your Favorite Ride!",
    description="Search for images of cars, trucks, buses, motorcycles, bicycles, airplanes, trains and boats ",

)

# Launch the interface
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://12094deccc0de60aff.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Evaluation

In [12]:
#  Ground Truth Evaluation Setup
ground_truth = {
    "car": ["img_13.jpg","img_14.jpg","img_17.jpg","img_20.jpg","img_21.jpg","img_71.jpg","img_84.jpg","img_85.jpg","img_88.jpg","img_89.jpg","img_91.jpg","img_93.jpg","img_98.jpg","img_108.jpg","img_109.jpg","img_110.jpg","img_115.jpg","img_121.jpg","img_123.jpg","img_124.jpg"],
    "truck": ["img_2.jpg","img_5.jpg","img_8.jpg","img_9.jpg","img_10.jpg","img_13.jpg","img_15.jpg","img_18.jpg","img_19.jpg","img_25.jpg","img_28.jpg","img_37.jpg","img_44.jpg","img_45.jpg","img_46.jpg","img_47.jpg","img_60.jpg","img_62.jpg","img_65.jpg","img_69.jpg"],
    "train": ["img_1.jpg","img_2.jpg","img_4.jpg","img_6.jpg","img_7.jpg","img_8.jpg","img_12.jpg","img_13.jpg","img_14.jpg","img_18.jpg","img_19.jpg","img_22.jpg","img_25.jpg","img_43.jpg","img_49.jpg","img_50.jpg","img_51.jpg","img_57.jpg","img_59.jpg","img_62.jpg"],
    "boat": ["img_4.jpg","img_13.jpg","img_16.jpg","img_27.jpg","img_28.jpg","img_31.jpg","img_32.jpg","img_35.jpg","img_37.jpg","img_41.jpg","img_42.jpg","img_44.jpg","img_46.jpg","img_47.jpg","img_50.jpg","img_51.jpg","img_54.jpg","img_59.jpg","img_60.jpg","img_61.jpg"],
    "motorcycle": ["img_0.jpg","img_1.jpg","img_7.jpg","img_11.jpg","img_12.jpg","img_17.jpg","img_18.jpg","img_21.jpg","img_22.jpg","img_24.jpg","img_25.jpg","img_31.jpg","img_39.jpg","img_44.jpg","img_46.jpg","img_48.jpg","img_54.jpg","img_55.jpg","img_56.jpg","img_58.jpg"],
    "bicycle": ["img_2.jpg","img_4.jpg","img_13.jpg","img_14.jpg","img_19.jpg","img_20.jpg","img_21.jpg","img_23.jpg","img_27.jpg","img_28.jpg","img_40.jpg","img_42.jpg","img_43.jpg","img_44.jpg","img_45.jpg","img_51.jpg","img_52.jpg","img_55.jpg","img_57.jpg","img_58.jpg"],
    "airplane": ["img_0.jpg","img_6.jpg","img_7.jpg","img_9.jpg","img_12.jpg","img_13.jpg","img_14.jpg","img_16.jpg","img_19.jpg","img_34.jpg","img_52.jpg","img_55.jpg","img_56.jpg","img_65.jpg","img_67.jpg","img_69.jpg","img_70.jpg","img_83.jpg","img_101.jpg","img_108.jpg"],
    "bus": ["img_0.jpg","img_4.jpg","img_10.jpg","img_12.jpg","img_27.jpg","img_28.jpg","img_34.jpg","img_44.jpg","img_49.jpg","img_51.jpg","img_53.jpg","img_55.jpg","img_59.jpg","img_75.jpg","img_76.jpg","img_77.jpg","img_80.jpg","img_81.jpg","img_94.jpg","img_8.jpg"]
}

def precision_at_k(retrieved, relevant, k=5):
    retrieved = retrieved[:k]
    return sum(1 for doc in retrieved if doc in relevant) / k

def recall_at_k(retrieved, relevant, k=5):
    if not relevant:
        return 0
    retrieved = retrieved[:k]
    return sum(1 for doc in retrieved if doc in relevant) / len(relevant)

def reciprocal_rank(retrieved, relevant):
    for rank, doc in enumerate(retrieved, start=1):
        if doc in relevant:
            return 1 / rank
    return 0.0

def get_top_docs(query_text, inverted_index, doc_lengths, top_n=5):
    bm25_scores = compute_bm25(query_text, inverted_index, doc_lengths)[:top_n]
    return [doc_id for doc_id, _ in bm25_scores]

def average_precision(retrieved, relevant, k=5):
    hits = 0
    sum_precisions = 0.0
    for i, doc in enumerate(retrieved[:k], start=1):
        if doc in relevant:
            hits += 1
            sum_precisions += hits / i
    return sum_precisions / len(relevant) if relevant else 0.0

def evaluate_ground_truth(ground_truth, top_k=5):
    precisions = []
    recalls = []
    mrrs = []
    aps = []
    accuracy_hits = 0  # Count of queries where at least one relevant doc is retrieved

    for query, relevant_docs in ground_truth.items():
        print(f"\n Evaluating query: {query}")
        preprocessed_query = preprocess_query(query)
        class_name = get_class_from_query(query)

        if not class_name:
            print("No class found in query.")
            continue

        csv_filename = f"{class_name}_image_captions.csv"
        preprocess_csv(csv_filename)
        build_inverted_index_tfidf(csv_filename)

        retrieved_docs = get_top_docs(preprocessed_query, inverted_index, doc_lengths, top_n=top_k)

        p = precision_at_k(retrieved_docs, relevant_docs, k=top_k)
        r = recall_at_k(retrieved_docs, relevant_docs, k=top_k)
        rr = reciprocal_rank(retrieved_docs, relevant_docs)
        ap = average_precision(retrieved_docs, relevant_docs, k=top_k)
        accuracy = 1 if any(doc in relevant_docs for doc in retrieved_docs[:top_k]) else 0

        print(f"Precision@{top_k}: {p:.2f}, Recall@{top_k}: {r:.2f}, MRR: {rr:.2f}, AP: {ap:.2f}, Accuracy: {accuracy}")

        precisions.append(p)
        recalls.append(r)
        mrrs.append(rr)
        aps.append(ap)
        accuracy_hits += accuracy

    num_queries = len(precisions)

    print("\nFinal Evaluation Results:")
    print(f"Average Precision@{top_k}: {sum(precisions)/num_queries:.2f}")
    print(f"Average Recall@{top_k}: {sum(recalls)/num_queries:.2f}")
    print(f"Mean Reciprocal Rank (MRR): {sum(mrrs)/num_queries:.2f}")
    print(f"Mean Average Precision (mAP): {sum(aps)/num_queries:.2f}")
    print(f"Accuracy@{top_k}: {accuracy_hits/num_queries:.2f}")


evaluate_ground_truth(ground_truth, top_k=5)


 Evaluating query: car
CSV file 'car_image_captions.csv' has been updated with combined caption and detection.
Building inverted index with TF-IDF...

Inverted Index with TF-IDF:
'bus': {'img_0.jpg': 0.3953127366441464, 'img_1.jpg': 0.1976563683220732, 'img_4.jpg': 0.3953127366441464, 'img_5.jpg': 0.1976563683220732, 'img_6.jpg': 0.3953127366441464, 'img_8.jpg': 0.3953127366441464, 'img_10.jpg': 0.3953127366441464, 'img_12.jpg': 0.3953127366441464, 'img_13.jpg': 0.1976563683220732, 'img_14.jpg': 0.3953127366441464, 'img_15.jpg': 0.13177091221471546, 'img_16.jpg': 0.1976563683220732, 'img_17.jpg': 0.1976563683220732, 'img_18.jpg': 0.3953127366441464, 'img_19.jpg': 0.3953127366441464, 'img_20.jpg': 0.23718764198648784, 'img_21.jpg': 0.2635418244294309, 'img_26.jpg': 0.06588545610735773, 'img_27.jpg': 0.3953127366441464, 'img_28.jpg': 0.3953127366441464, 'img_29.jpg': 0.1976563683220732, 'img_32.jpg': 0.06588545610735773, 'img_33.jpg': 0.1976563683220732, 'img_34.jpg': 0.3953127366441464

<ipython-input-9-bdd3ab141ab4>:78: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'detection'] = ""  # Clear the 'detection' column as it's now combined in 'caption'
<ipython-input-9-bdd3ab141ab4>:78: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'detection'] = ""  # Clear the 'detection' column as it's now combined in 'caption'
<ipython-input-9-bdd3ab141ab4>:78: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'detection'] = ""  # C

CSV file 'boat_image_captions.csv' has been updated with combined caption and detection.
Building inverted index with TF-IDF...

Inverted Index with TF-IDF:
'bus': {'img_0.jpg': 3.283414346005772, 'img_1.jpg': 0.1976563683220732, 'img_4.jpg': 0.3953127366441464, 'img_5.jpg': 0.1976563683220732, 'img_6.jpg': 0.3953127366441464, 'img_8.jpg': 0.3953127366441464, 'img_10.jpg': 0.3953127366441464, 'img_12.jpg': 0.3953127366441464, 'img_13.jpg': 0.1976563683220732, 'img_14.jpg': 0.3953127366441464, 'img_15.jpg': 0.13177091221471546, 'img_16.jpg': 0.1976563683220732, 'img_17.jpg': 0.1976563683220732, 'img_18.jpg': 0.3953127366441464, 'img_19.jpg': 0.3953127366441464, 'img_20.jpg': 0.23718764198648784, 'img_21.jpg': 0.9649969012508197, 'img_26.jpg': 0.06588545610735773, 'img_27.jpg': 0.3953127366441464, 'img_28.jpg': 0.3953127366441464, 'img_29.jpg': 0.1976563683220732, 'img_32.jpg': 0.06588545610735773, 'img_33.jpg': 0.1976563683220732, 'img_34.jpg': 0.3953127366441464, 'img_35.jpg': 0.263541

<ipython-input-9-bdd3ab141ab4>:78: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'detection'] = ""  # Clear the 'detection' column as it's now combined in 'caption'
<ipython-input-9-bdd3ab141ab4>:78: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'detection'] = ""  # Clear the 'detection' column as it's now combined in 'caption'
<ipython-input-9-bdd3ab141ab4>:78: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'detection'] = ""  # C